In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# For connection to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-31 02:45:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2021-12-31 02:45:12 (6.09 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("level1software").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [6]:
# Read in data from AWS and combine data into one DataFrame
from pyspark import SparkFiles
db_url = "jdbc:postgresql://<insert endpoint>:5432/big_data_challenge_l1_vg"
db_properties = {"user":"root", "password":"<insert password>", "driver":"org.postgresql.Driver"}
df = spark.read.jdbc(url=db_url, table='vine_table', properties=db_properties)

db_url = "jdbc:postgresql://<insert endpoint>:5432/big_data_challenge_l1_soft"
df2 = spark.read.jdbc(url=db_url, table='vine_table', properties=db_properties)

In [7]:
# Check on number of rows for each dataframe before uniting them into one
df.count()

1785997

In [8]:
# Check on number of rows for each dataframe before uniting them into one
df2.count()

341931

In [9]:
# Unite the two DataFrames into one then show the number of rows
df = df.union(df2)
df.count()

2127928

In [10]:
# Show DataFrame
df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1GBJZ5XC1XTXS|          5|            0|          0|   N|
| RFPA1A1M29NFW|          2|            2|          2|   N|
|R2S56QV606LUAD|          1|            0|          1|   N|
|R2RQUJDDM7MGIP|          5|            0|          0|   N|
|R3ANJRZZ5OHEO4|          5|            0|          0|   N|
| RID7CBMW3K5H0|          3|            0|          0|   N|
|R3EZD1NNJYO17V|          4|            0|          0|   N|
| RZ1BSNELR5NXZ|          5|            0|          0|   N|
| RX8S6DT16KEGN|          5|            0|          0|   N|
|R2DQ3PPS9I9XWU|          3|            0|          0|   N|
|R3FO35DQ9C0LHK|          5|            0|          0|   N|
|R39029SL4HEL3K|          5|            0|          0|   N|
|R378QYJF55FXXO|          4|            0|          0|   N|
|R3BF9ZNWIK2AVA|          2|            

In [11]:
# Remove duplicate rows and null values then show the number of rows
df = df.distinct()
df = df.na.drop()
df.count()

2127928

In [13]:
# Show current data types to verify these are still correct
df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

In [14]:
# Load the DataFrames that correspond to table into an RDS instance
mode = "overwrite"
db_url = "jdbc:postgresql://<insert endpoint>:5432/big_data_challenge"
db_properties = {"user":"root", "password":"<insert password>", "driver":"org.postgresql.Driver"}

df.write.jdbc(url=db_url, table='vine_table', mode=mode, properties=db_properties)